In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [82]:
df = pd.read_excel("C:/Users/gvrk1/Downloads/EV CLEAN DATASET.xlsx")

In [83]:
df.head()

,S.NO,FUEL,COLOR,VEHICLE CLASS CATEGORY,CATEGORY,BRAND,RTO-OFFICE,DISTRICT,REG.DATE,YEAR,MONTH
0,1,EV,ADAPT GREEN,3W Electric,Transport,ADAPT,RTA MANCHERIAL,Mancherial,2019-01-01,2019,January
1,2,EV,ADAPT GREEN,3W Electric,Transport,ADAPT,RTA NAGARKURNOOL,Nagarkurnool,2019-01-01,2019,January
2,3,EV,MEGNA GREY,2W,Non Transport,OKINAWA,RTA RANGAREDDY,Ranga Reddy,2019-01-01,2019,January
3,4,EV,ADAPT GREEN,3W Electric,Transport,ADAPT,RTA MANCHERIAL,Mancherial,2019-01-02,2019,January
4,5,EV,MEGNA GREY,2W,Non Transport,OKINAWA,RTA RANGAREDDY,Ranga Reddy,2019-01-02,2019,January


In [84]:
# Known inconsistent district names
district_mapping = {
    'Armoor? Bhudan? (Likely Armoor sub-office, Nizamabad)': 'Armoor (Nizamabad)',
    'Bhadrachalam (Bhadradri sub-agency)': 'Bhadrachalam',
    'Charminar Zone, GHMC (Demo)': 'Charminar Zone',
    'Hyderabad (Demo)': 'Hyderabad',
    'Medchal–Malkajgiri': 'Medchal-Malkajgiri',
    # Add more as needed
}

# Apply the replacement
df['DISTRICT'] = df['DISTRICT'].replace(district_mapping)


In [85]:
# Group and count EVs per DISTRICT and YEAR
ev_counts = df.groupby(['DISTRICT', 'YEAR']).size().reset_index(name='EV_COUNT')

# Check result
ev_counts.head(10)

,DISTRICT,YEAR,EV_COUNT
0,Adilabad,2019,27
1,Adilabad,2020,14
2,Adilabad,2021,171
3,Adilabad,2022,271
4,Adilabad,2023,387
5,Adilabad,2024,533
6,Adilabad,2025,252
7,Armoor (Nizamabad),2019,2
8,Armoor (Nizamabad),2020,113
9,Armoor (Nizamabad),2021,633


In [86]:
# Pivot the EV count table
ev_pivot = ev_counts.pivot(index='DISTRICT', columns='YEAR', values='EV_COUNT').fillna(0).reset_index()

# Check the pivoted table
ev_pivot.head()


YEAR,DISTRICT,2019,2020,2021,2022,2023,2024,2025
0,Adilabad,27.0,14.0,171.0,271.0,387.0,533.0,252.0
1,Armoor (Nizamabad),2.0,113.0,633.0,921.0,1045.0,1212.0,620.0
2,Bhadrachalam,0.0,10.0,5.0,51.0,136.0,273.0,105.0
3,Bhadradri Kothagudem,2.0,4.0,25.0,143.0,402.0,652.0,295.0
4,Hanamkonda,0.0,0.0,111.0,525.0,1245.0,1513.0,716.0


In [87]:
df.columns

Index(['S.NO', 'FUEL', 'COLOR', 'VEHICLE CLASS CATEGORY', 'CATEGORY', 'BRAND',
       'RTO-OFFICE', 'DISTRICT', 'REG.DATE', 'YEAR', 'MONTH'],
      dtype='object')

In [88]:
# Add a column to identify Jan–June data
df['MONTH'] = pd.to_datetime(df['REG.DATE']).dt.month
df['HALF'] = df['MONTH'].apply(lambda x: 'H1' if x <= 6 else 'H2')

# Calculate H1 and full year counts per year
ev_half_year = df.groupby(['DISTRICT', 'YEAR', 'HALF']).size().unstack(fill_value=0).reset_index()
ev_half_year['FULL_YEAR'] = ev_half_year['H1'] + ev_half_year['H2']
ev_half_year['H1_RATIO'] = ev_half_year['H1'] / ev_half_year['FULL_YEAR']

# Compute average H1_RATIO across districts for each year (excluding 2025)
avg_h1_ratios = ev_half_year[ev_half_year['YEAR'] < 2025].groupby('YEAR')['H1_RATIO'].mean()
avg_ratio = avg_h1_ratios.mean()
print(f"Average Jan–June ratio (past years): {avg_ratio:.2f}")


Average Jan–June ratio (past years): 0.48


In [89]:
# Filter 2025 Jan–June data
df_2025 = df[(df['YEAR'] == 2025) & (df['MONTH'] <= 6)]

# Get Jan–June EV counts per district
ev_2025_h1 = df_2025.groupby('DISTRICT').size().reset_index(name='H1_COUNT')

# Estimate full-year count
ev_2025_h1['FULL_2025_ESTIMATE'] = (ev_2025_h1['H1_COUNT'] / avg_ratio).round().astype(int)

# Create a 2025 dataframe compatible with ev_pivot
ev_2025_full = ev_2025_h1[['DISTRICT', 'FULL_2025_ESTIMATE']]
ev_2025_full.columns = ['DISTRICT', 2025]


In [90]:
# Remove 2025 if it already exists in ev_pivot
if 2025 in ev_pivot.columns:
    ev_pivot.drop(columns=[2025], inplace=True)

# Merge
ev_pivot = ev_pivot.merge(ev_2025_full, on='DISTRICT', how='left')
ev_pivot[2025] = ev_pivot[2025].fillna(0).astype(int)


In [91]:
# Step 1: Create a copy
df_ml = ev_pivot.copy()

# Step 2: Generate features (lag values and growth trends)
for year in range(2020, 2026):
    df_ml[f'growth_{year}'] = df_ml[year] - df_ml[year - 1]

# Step 3: Average growth
df_ml['avg_growth'] = df_ml[[f'growth_{year}' for year in range(2020, 2026)]].mean(axis=1)

# Step 4: Use last 3 years as individual features
df_ml['last_count'] = df_ml[2025]
df_ml['count_2024'] = df_ml[2024]
df_ml['count_2023'] = df_ml[2023]

# Step 5: Set target as avg_growth + 2025
df_ml['target_2026'] = df_ml['last_count'] + df_ml['avg_growth']

# Final feature set
features = ['count_2023', 'count_2024', 'last_count', 'avg_growth']
X = df_ml[features]
y = df_ml['target_2026']


In [92]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
model = LinearRegression()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("R² Score:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))


R² Score: 1.0
RMSE: 1.2036518443130546e-12


C:\Users\gvrk1\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [93]:
df_ml['predicted_2026'] = model.predict(X).round().astype(int)


In [94]:
# Top 10 districts by predicted EV count in 2026
top_10_districts = df_ml[['predicted_2026']].sort_values(by='predicted_2026', ascending=False).head(10)
top_10_districts.reset_index(inplace=True)
top_10_districts.rename(columns={'index': 'DISTRICT'}, inplace=True)

top_10_districts



,DISTRICT,predicted_2026
0,41,21266
1,6,8119
2,27,7510
3,47,6571
4,8,4536
5,22,3290
6,10,3222
7,9,3177
8,36,2840
9,7,2467


In [95]:
# Sort df_ml by predicted EVs for 2026
top_10_districts = df_ml.sort_values(by='predicted_2026', ascending=False).head(10)

# Select only the columns you want to display
top_10_districts = top_10_districts[['DISTRICT', 'predicted_2026']]

top_10_districts


,DISTRICT,predicted_2026
41,Ranga Reddy,21266
6,Hyderabad Central,8119
27,Medchal-Malkajgiri,7510
47,Uppal (Hyderabad East area),6571
8,Hyderabad North,4536
22,Kukatpally (Hyderabad-urban),3290
10,Hyderabad West,3222
9,Hyderabad South,3177
36,Patancheruvu (Ranga Reddy),2840
7,Hyderabad East,2467
